In [3]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

In [5]:
import pandas as pd
import datetime as dt

data = pd.read_csv("../raw_data/stock_prices_latest.csv",parse_dates = True)
data.head()

,symbol,date,open,high,low,close,close_adjusted,volume,split_coefficient
0,MSFT,2016-05-16,50.80,51.96,50.75,51.83,49.7013,20032017,1.0
1,MSFT,2002-01-16,68.85,69.84,67.85,67.87,22.5902,30977700,1.0
2,MSFT,2001-09-18,53.41,55.00,53.17,54.32,18.0802,41591300,1.0
3,MSFT,2007-10-26,36.01,36.03,34.56,35.03,27.2232,288121200,1.0
4,MSFT,2014-06-27,41.61,42.29,41.51,42.25,38.6773,74640000,1.0


In [6]:
data['date'] = pd.to_datetime(data['date'])

In [7]:
data_two_years = data[(data.date.dt.year == 2017) | (data.date.dt.year == 2018)]
data_two_years

,symbol,date,open,high,low,close,close_adjusted,volume,split_coefficient
8,MSFT,2017-05-08,68.97,69.05,68.420,68.94,67.8482,18446053,1.0
22,MSFT,2017-12-27,85.65,85.98,85.215,85.71,85.7100,13000828,1.0
24,MSFT,2017-05-01,68.68,69.55,68.500,69.41,68.3107,31304672,1.0
33,MSFT,2017-09-18,75.23,75.97,75.040,75.16,74.7815,22730355,1.0
43,MSFT,2017-09-07,73.68,74.60,73.600,74.34,73.9656,17165518,1.0
...,...,...,...,...,...,...,...,...,...
23300476,PAEWW,2018-11-06,1.50,1.50,1.500,1.50,1.5000,12000,1.0
23300481,PAEWW,2018-12-12,1.20,1.20,1.200,1.20,1.2000,75000,1.0
23300491,PAEWW,2018-12-18,1.30,1.30,1.300,1.30,1.3000,15000,1.0
23300495,PAEWW,2018-11-29,1.20,1.40,1.200,1.40,1.4000,10000,1.0


In [9]:
data_two_years = data_two_years[5::6]
data_two_years

,symbol,date,open,high,low,close,close_adjusted,volume,split_coefficient
512,MSFT,2017-01-19,62.24,62.98,62.195,62.30,60.9452,18451655,1.0
1205,MSFT,2017-06-15,69.27,70.21,68.800,69.90,69.1795,25701569,1.0
1670,MSFT,2017-03-21,65.19,65.50,64.130,64.21,63.1931,26640480,1.0
2369,MSFT,2017-06-29,69.38,69.49,68.090,68.49,67.7840,28231562,1.0
3014,MSFT,2017-06-30,68.78,69.38,68.740,68.93,68.2195,23039328,1.0
...,...,...,...,...,...,...,...,...,...
23299507,XAIR,2017-12-19,5.00,5.00,4.750,4.76,4.7600,3678,1.0
23299595,XAIR,2018-08-29,3.60,3.65,3.600,3.60,3.6000,1500,1.0
23299678,XAIR,2018-08-31,3.60,3.60,3.590,3.59,3.5900,2000,1.0
23299761,XAIR,2018-03-13,3.65,3.65,3.500,3.60,3.6000,5007,1.0


In [16]:
date_time = pd.to_datetime(data_two_years.pop('date'), format='%Y.%m.%d')

In [17]:
data_two_years.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
open,88707.0,78.765294,3.524396e+03,0.00,7.84,18.80,39.05000,3.303000e+05
high,88707.0,79.439407,3.548153e+03,0.00,7.98,19.05,39.59310,3.309904e+05
low,88707.0,77.959572,3.491032e+03,0.00,7.67,18.54,38.55000,3.273200e+05
close,88707.0,78.812010,3.527743e+03,0.00,7.83,18.80,39.06000,3.279900e+05
close_adjusted,88707.0,88.548614,3.578805e+03,0.00,8.13,18.90,39.20725,3.279900e+05
volume,88707.0,899679.101232,3.713534e+06,0.00,23990.50,140083.00,611205.50000,4.232506e+08
split_coefficient,88707.0,0.999917,8.741741e-03,0.04,1.00,1.00,1.00000,1.250000e+00


In [18]:
column_indices = {name: i for i, name in enumerate(data_two_years.columns)}

n = len(data_two_years)
train_df = data_two_years[0:int(n*0.7)]
val_df = data_two_years[int(n*0.7):int(n*0.9)]
test_df = data_two_years[int(n*0.9):]

num_features = data_two_years.shape[1]


In [19]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

In [30]:
class prediction():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])


In [31]:
w1 = prediction(input_width=24, label_width=1, shift=24,
                     label_columns=['T (degC)'])     #à modifier
w1


Total window size: 48
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
Label indices: [47]
Label column name(s): ['T (degC)']

In [ ]:
data_two_years = data[(data.date.dt.year == 2017) | (data.date.dt.year == 2018)]
data_two_years